#### Importing Necessary Libraries

In [15]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone.grpc import PineconeGRPC as Pinecone
import os
import json
from datetime import datetime

#### Initializing and Accessing Pinecone Database and OpenAI

In [16]:
def initialize_components(openai_api_key, pinecone_api_key, index_name):
    """Initialize and return the Pinecone index, vectorstore, and language model."""
    pc = Pinecone(api_key=pinecone_api_key)
    index = pc.Index(index_name)

    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="text")
    
    llm = ChatOpenAI(temperature=0.1, model="gpt-4", openai_api_key=openai_api_key)
    return index, vectorstore, llm

#### Retriving relavant context based on the given questions

In [17]:
def get_context(vectorstore, question, k=4):
    """Retrieve relevant context based on the given question."""
    results = vectorstore.similarity_search_with_score(question, k=k)
    return "\n".join([doc.page_content for doc, _ in results])

#### Function to validate the answer

In [18]:
def validate_response(response, options):
    """
    Validate that the response contains one of the valid options (A, B, C, D).
    If no valid answer is found, return a warning message.
    """
    valid_answers = ["A", "B", "C", "D"]
    for answer in valid_answers:
        if f"Final Answer: {answer}" in response:
            return response
    return "Invalid response: No valid answer found. Please check the reasoning."

#### Chain-Of-Thought (COT) Reasoning

In [19]:
def chain_of_thought_reasoning(llm, question, options, context):
    """Perform chain-of-thought reasoning."""
    prompt = """
    Use chain-of-thought reasoning to solve this medical question. Analyze the context step by step and choose the most appropriate answer:

    Question: {question}

    Options:
    {options}

    Relevant Context:
    {context}

    Let's solve this step by step:
    1. Identify the key patient information and medical condition.
    2. Consider relevant medical guidelines and best practices from the context.
    3. Evaluate each option against the guidelines and the patient's specific situation.
    4. Choose the most appropriate answer from the options.

    Provide your reasoning and final answer in this format:
    Reasoning Chain:
    [Step-by-step reasoning]

    Final Answer: [Selected option letter]
    Confidence: [High/Medium/Low]
    Justification: [Brief explanation of final choice]
    """
    messages = [{"role": "user", "content": prompt.format(
        question=question,
        options="\n".join(options),
        context=context
    )}]
    response = llm.invoke(messages).content
    return validate_response(response, options)


#### Tree-Of-Thought Reasoning

In [20]:
def tree_of_thought_reasoning(llm, question, options, context):
    """Perform tree-of-thought reasoning."""
    prompt = """
    Use tree-of-thought reasoning to explore different decision paths for this medical question. Analyze the context, explore various paths, and choose the most appropriate answer:

    Question: {question}

    Options:
    {options}

    Relevant Context:
    {context}

    Explore multiple reasoning paths:
    1. Safety Considerations Path:
       - What are the safety implications of each option?
       - What are the potential risks and contraindications?

    2. Clinical Guidelines Path:
       - Which options align with standard medical protocols?
       - What do the guidelines recommend for this situation?

    3. Patient-Specific Factors Path:
       - How do the patient's specific circumstances affect each option?
       - What are the individual risk factors and considerations?

    After analyzing each path, select the best answer from the options.

    Provide your analysis and final answer in this format:
    Path 1 (Safety) Analysis:
    [Analysis]

    Path 2 (Guidelines) Analysis:
    [Analysis]

    Path 3 (Patient-Specific) Analysis:
    [Analysis]

    Combined Decision:
    Final Answer: [Selected option letter]
    Confidence: [High/Medium/Low]
    Justification: [Brief explanation combining insights from all paths]
    """
    messages = [{"role": "user", "content": prompt.format(
        question=question,
        options="\n".join(options),
        context=context
    )}]
    response = llm.invoke(messages).content
    return validate_response(response, options)


#### Structural Medical Reasoning

In [21]:
def structured_medical_reasoning(llm, question, options, context):
    """Perform structured medical reasoning using the SOAP format."""
    prompt = """
    Apply structured medical reasoning using the SOAP format to analyze this case and choose the most appropriate answer:

    Question: {question}

    Options:
    {options}

    Relevant Context:
    {context}

    Provide your analysis in SOAP format and make a final decision:

    Subjective:
    - Chief complaint
    - Relevant history
    - Patient's current situation

    Objective:
    - Key findings from the case
    - Relevant data from context
    - Evidence-based guidelines applicable

    Assessment:
    - Analysis of the medical situation
    - Evaluation of each treatment option
    - Risk-benefit analysis

    Plan:
    Final Answer: [Selected option letter]
    Confidence: [High/Medium/Low]
    Justification: [Medical reasoning for the chosen plan]
    """
    messages = [{"role": "user", "content": prompt.format(
        question=question,
        options="\n".join(options),
        context=context
    )}]
    response = llm.invoke(messages).content
    return validate_response(response, options)


#### Running the Experiment for each reasoning Process and Saving the Result

In [22]:
def run_experiment(openai_api_key, pinecone_api_key, index_name, question, options):
    """Run all reasoning methods and save the results."""
    index, vectorstore, llm = initialize_components(openai_api_key, pinecone_api_key, index_name)
    context = get_context(vectorstore, question)

    experiments = {
        "chain_of_thought": {
            "method": "Chain-of-thought reasoning",
            "response": chain_of_thought_reasoning(llm, question, options, context)
        },
        "tree_of_thought": {
            "method": "Tree-of-thought reasoning",
            "response": tree_of_thought_reasoning(llm, question, options, context)
        },
        "structured_medical": {
            "method": "Structured medical reasoning (SOAP)",
            "response": structured_medical_reasoning(llm, question, options, context)
        }
    }

    results = {
        "question": question,
        "options": options,
        "context": context,
        "experiments": experiments
    }

    # Save results to file
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"reasoning_experiments_{timestamp}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print(f"\nResults saved to: {filename}")
    return results

#### Main Function

In [23]:
def main():
    """Main function to run the reasoning experiment."""
    sample_mcq = {
        "question": "A 32-year-old male patient with a history of substance abuse disorder (heroin) has presented with severe toothache for the last three days. His medical history reveals he is in active treatment for substance use disorder and he has been maintaining a drug-free lifestyle for the past six months. He's currently having a severe dental abscess in the lower right second molar confirmed by dental radiography. How would you manage his condition?",
        "options": [
            "A) Prescribe opioids for pain relief",
            "B) Consult with the patient's addiction specialist before prescribing opioids for pain management",
            "C) Prescribe non-opioid analgesics and refer the patient to a dental surgeon for abscess management",
            "D) Ignore the patient's request for pain relief due to his substance abuse history"
        ]
    }

    # Run the experiment
    results = run_experiment(
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        pinecone_api_key=os.getenv("PINECONE_API_KEY"),
        index_name="medicalqabot",
        question=sample_mcq["question"],
        options=sample_mcq["options"]
    )

    # Print summary
    print("\nExperiment Summary:")
    for method, data in results["experiments"].items():
        print(f"\n{method.replace('_', ' ').title()}:")
    
        response = data.get("response", "")
        print(f"Response:\n{response[:500]}...")  



In [24]:
if __name__ == "__main__":
    main()


Results saved to: reasoning_experiments_20250112_230844.json

Experiment Summary:

Chain Of Thought:
Response:
Reasoning Chain:
1. The patient is a 32-year-old male with a history of substance abuse disorder (heroin) and is currently in active treatment for substance use disorder. He has been maintaining a drug-free lifestyle for the past six months. He is presenting with severe toothache due to a dental abscess in the lower right second molar.
2. The Dental Guideline on Prescribing Opioids for Acute Pain Management advises against prescribing opioids for patients with a history of substance abuse disord...

Tree Of Thought:
Response:
Path 1 (Safety) Analysis:
Option A (prescribing opioids) poses a significant risk due to the patient's history of substance abuse. Option D (ignoring the patient's request for pain relief) is not safe or ethical as it neglects the patient's immediate need for pain management. Options B and C are safer as they either involve consultation with the patient'